In [ ]:
#########################################
#           Python Version              #
# 3.7.7 (default, May  6 2020, 04:59:01)# 
# [Clang 4.0.1 (tags/RELEASE_401/final)]#
#########################################

In [ ]:
from docx import Document  
from docx.oxml.ns import qn  
from docx.shared import Pt, RGBColor

In [ ]:
def formatQuoteAt(lst):
    k = []
    for j in range(len(lst)):
        if "CQ:quote" in lst[j]:
            continue
        elif "CQ:at" in lst[j]:
            at_start = lst[j].find("@")
            lst[j] = lst[j][at_start:]
            at_end = lst[j].find("&")
            lst[j] = lst[j][:at_end] + "\t"
            k.append(lst[j])
        elif "CQ:At" in lst[j]:
            at_start = lst[j].find("[")
            lst[j] = lst[j][:at_start]
            k.append(lst[j])
        else:
            k.append(lst[j])
    lst = [k[0]]
    for p in range(1,len(k)):
        lst = appendNonRepeat(lst, k[p])
    return lst

In [ ]:
def appendNonRepeat(clean, strip):
    if (strip not in clean[-1]):
        clean.append(strip)
    return clean

In [ ]:
def extractAlias(text):
    aliasLst = []
    cardNames = text.split("]:")
    for cardName in cardNames[:-1]:
        cardName = cardName.split('[')[-1]
        aliasLst.append(cardName)
    atNames = text.split("&#93;.display]")
    for atName in atNames[:-1]:
        atName = atName.split('@')[-1]
        aliasLst.append(atName)
    aliasLst = list(set(aliasLst))
    alias = {}
    for key in aliasLst:
        alias[key] = ''
    return alias

In [ ]:
def enTransToC(string):
    enPun = u',~!?()"\''
    cPun = u'，～！？（）“‘'
    table= {ord(f):ord(t) for f,t in zip(enPun,cPun)}
    return string.translate(table)

In [ ]:
def replaceAlias(text, alias):
    for key in alias.keys():
        text = text.replace(key, alias[key])
    return text

In [ ]:
def removeCQBot(textLst):
    textLst = [i for i in textLst if ("msg brief" not in i) and ("CQ:image" not in i) and ("切换到人物卡" not in i) and ("CQ:service" not in i)]
    return textLst

In [ ]:
# read file and draft alias
def stepOne(path, encoding):
    # read file
    with open(path, encoding=encoding) as f:
        text = f.read()
    # extract dictionary
    print(extractAlias(text))
    return text

In [ ]:
# split and clean by line 
def stepTwo(text, alias):
    # replace alias
    text = replaceAlias(text, alias)
    
    # split to list
    textLst = text.split("\n")
    # remove image and reference
    textLst = removeCQBot(textLst)
    textLst = [i.strip() for i in textLst]
    textLst = [i for i in textLst if i]

    # clean by line    
    clean = ['']
    strip = ''
    for i in textLst:
        # detect start of a message
        if (i[0] == '<'):
            clean = appendNonRepeat(clean, strip)
            print(i)
            i = i.split("]")
            print("split", i)
            # detect bot message(with bot name and player name)
            if (len(i)==3 and "bot" in i[0]):
                i[1] = i[1] + "]"
                i[-1] = i[-1] + " "
            # detect quote and at
            elif (len(i)>2):
                i = formatQuoteAt(i)
                if ':\t' != i[1][:2]:
                    i[0] = i[0] +':\t'
            print("whyyy", i)
            i[0] = i[0].split("[")[1]
            strip = "".join(i)
        else:
            i = i.split("]")
            i = formatQuoteAt(i)
            i = "".join(i)
            strip = strip + i
    clean = appendNonRepeat(clean, strip)
    clean = "\n".join(clean[1:])
    
    # fix punctuations
    clean = enTransToC(clean)
    
    return clean

In [ ]:
# encoding and write file
def stepThree(clean, encoding, path):
    clean.encode(encoding)
    with open(path, "w", encoding=encoding) as f:
        f.write(clean)

In [ ]:
def removePare(text):
    clean = []
    marked = []
    for i in text:
        i = i.strip()
        j = i.split('\t')
        if (len(j) == 1):
            continue
        elif ("（" == j[1][0]) \
            or ("@" in i and "）" in i[-1]) \
            or (("（" not in i) and ("）" in i[-1])):
            marked.append("pare:\t"+i)
            continue
        elif ("（" in i and "）" not in i):
            i = "".join(i.split("（")[:-1])
            marked.append(i)
            marked.append("pare:\t"+i)
            clean.append(i)
        else:
            marked.append(i)
            clean.append(i)
    clean = "\n".join(clean)
    return marked, clean

def stepFour(path, encoding):
    with open(path, encoding=encoding) as f:
        text = f.readlines()    
    clean = removePare(text)
    return clean

In [ ]:
def extractPalette(textLst):
    names = []
    for i in textLst:
        names.append(i[0])
    names = list(set(names))
    palette = {}
    for key in names:
        palette[key] = (0x0, 0x0, 0x0)
    palette["pare"] = (241, 220, 167)
    return palette

def stepFive(text):
    textLst = text.split("\n")
    textLstSplit = [i.split(":\t") for i in textLst]
    print(extractPalette(textLstSplit))
    return textLst

In [ ]:
def writeBleached(textLst, path, font, fontSize):
    document = Document()
    document.styles['Normal'].font.name = font  
    document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), font)  
    document.styles['Normal'].font.size = Pt(fontSize) 
    paragraph = document.add_paragraph()
    textLstSplit = [i.split(":\t") for i in textLst]
    for i in textLstSplit:
        run = paragraph.add_run(':\t'.join(i)+'\n')
        rgb = palette[i[0]]
        run.font.color.rgb = RGBColor(rgb[0], rgb[1], rgb[2])
    document.save(path)
    
def writeRaw(textLst, path, font, fontSize):
    document = Document()
    document.styles['Normal'].font.name = font  
    document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), font)  
    document.styles['Normal'].font.size = Pt(fontSize) 
    paragraph = document.add_paragraph()
    textLstSplit = [i.split(":\t") for i in textLst]
    for i in textLstSplit:
        if i[0] == "pare":
            run = paragraph.add_run(':\t'.join(i[1:])+'\n')
            rgb = palette[i[0]]
            run.font.color.rgb = RGBColor(rgb[0], rgb[1], rgb[2])
        else:
            run = paragraph.add_run(':\t'.join(i)+'\n')
            rgb = palette[i[0]]
            run.font.color.rgb = RGBColor(rgb[0], rgb[1], rgb[2])
    document.save(path)
    
def stepSix(textLstRaw, textLstBleached, projectPath, toPath, font, fontSize):
    pathRaw = projectPath + 'RAW-' + toPath.split('.')[0] + '.docx'
    writeRaw(textLstRaw, pathRaw, font, fontSize)
    pathBleached = projectPath + 'BLEACHED-' + toPath.split('.')[0] + '.docx'
    writeBleached(textLstBleached, pathBleached, font, fontSize)

In [ ]:
#####################
#    START HERE     #
#####################

###########################
#    PHASE I: CLEANING    #
###########################

In [ ]:
PROJECT_PATH = "log/logname/"
FROM_PATH = "《logname》原文件.txt"
TO_PATH = "logname.txt"

FROM_ENCODING = "gbk"
TO_ENCODING = "utf-8"

FONT = 'FangSong'
FONTSIZE = 10.5

# read file and draft alias
text = stepOne(PROJECT_PATH+FROM_PATH, FROM_ENCODING)

In [ ]:
# fill the correct alias
alias = {'骰娘': 'bot'}

In [ ]:
# clean by line
clean = stepTwo(text, alias)

# check result in jupyter notebook
print(clean)

In [ ]:
# encoding and write file
stepThree(clean, TO_ENCODING, PROJECT_PATH+'RAW-'+TO_PATH)

In [ ]:
###############################
#    PHASE II: REMOVE PARE    #
###############################

In [ ]:
# remove parentheses and review
marked, clean = stepFour(PROJECT_PATH+'RAW-'+TO_PATH, TO_ENCODING)
print(clean)

In [ ]:
# encoding and write file
stepThree(clean, TO_ENCODING, PROJECT_PATH+'BLEACHED-'+TO_PATH)

In [ ]:
###############################
#      PHASE III: DYEING      #
###############################

In [ ]:
clean = stepFive(clean)

In [ ]:
palette = {'bot': (255, 201, 92), 
           'pare': (185, 177, 155)
          }

In [ ]:
stepSix(marked, clean, PROJECT_PATH, TO_PATH, FONT, FONTSIZE)